In [1]:
import os
import argparse
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from PIL import Image

from torchvision.models import resnet50, ResNet50_Weights

In [2]:
num_workers = 8
gpu = True
epochs = 1

In [7]:
device = torch.device("mps")
device.type

'mps'

In [8]:
torch.cuda.is_available()

False

In [9]:
class ResNet50Food(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = ResNet50_Weights.IMAGENET1K_V2
        self.pretrained = resnet50(weights=self.weights)
        self.fc2 = nn.Linear(in_features=1000, out_features=101, bias=True)
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.fc2(x)
        return x
    
net = ResNet50Food()
#net.train()

In [13]:
net.parameters()

<generator object Module.parameters at 0x15cd53b50>

In [14]:
for p in net.pretrained.parameters():
    print(p)

Parameter containing:
tensor([[[[-7.4457e-03, -3.1783e-03,  3.7353e-02,  ...,  4.7936e-02,
           -2.0325e-02,  8.8140e-03],
          [-5.7435e-02,  4.4709e-02,  7.7509e-02,  ...,  8.8442e-02,
            2.9346e-02, -5.8331e-02],
          [ 6.8356e-02, -2.7044e-01,  4.0348e-01,  ..., -1.6491e-01,
            2.1868e-01, -7.2909e-02],
          ...,
          [-1.0874e-01,  3.8148e-01, -4.5487e-01,  ...,  6.8366e-01,
           -5.7855e-01,  2.2461e-01],
          [ 2.5698e-02, -1.7703e-01,  6.4375e-01,  ...,  5.2644e-01,
           -4.9317e-02, -6.8082e-02],
          [ 4.5281e-02, -1.3072e-01,  1.7864e-02,  ..., -3.5753e-01,
            1.8976e-01, -2.2302e-02]],

         [[ 8.9197e-03,  4.8768e-03, -1.5356e-02,  ...,  8.6949e-02,
           -6.5541e-02,  1.6895e-02],
          [-3.3543e-02,  3.9067e-02,  8.6565e-02,  ...,  1.1713e-01,
            2.1848e-02, -4.4082e-02],
          [ 5.5285e-02, -2.6424e-01,  4.2687e-01,  ..., -2.3723e-01,
            3.2017e-01, -1.1832e-01]

In [15]:
for p in net.pretrained.parameters():
    p.requires_grad = False

In [16]:
for p in net.pretrained.parameters():
    print(p)

Parameter containing:
tensor([[[[-7.4457e-03, -3.1783e-03,  3.7353e-02,  ...,  4.7936e-02,
           -2.0325e-02,  8.8140e-03],
          [-5.7435e-02,  4.4709e-02,  7.7509e-02,  ...,  8.8442e-02,
            2.9346e-02, -5.8331e-02],
          [ 6.8356e-02, -2.7044e-01,  4.0348e-01,  ..., -1.6491e-01,
            2.1868e-01, -7.2909e-02],
          ...,
          [-1.0874e-01,  3.8148e-01, -4.5487e-01,  ...,  6.8366e-01,
           -5.7855e-01,  2.2461e-01],
          [ 2.5698e-02, -1.7703e-01,  6.4375e-01,  ...,  5.2644e-01,
           -4.9317e-02, -6.8082e-02],
          [ 4.5281e-02, -1.3072e-01,  1.7864e-02,  ..., -3.5753e-01,
            1.8976e-01, -2.2302e-02]],

         [[ 8.9197e-03,  4.8768e-03, -1.5356e-02,  ...,  8.6949e-02,
           -6.5541e-02,  1.6895e-02],
          [-3.3543e-02,  3.9067e-02,  8.6565e-02,  ...,  1.1713e-01,
            2.1848e-02, -4.4082e-02],
          [ 5.5285e-02, -2.6424e-01,  4.2687e-01,  ..., -2.3723e-01,
            3.2017e-01, -1.1832e-01]

In [ ]:
transform = transforms.Compose(
        [transforms.Resize(256,antialias=True),
        transforms.CenterCrop(224),
        #transforms.ConvertImageDtype(torch.float32),
        transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

preprocess = net.weights.transforms()

data = torchvision.datasets.Food101(root="./data",
                                    split="train",
                                    transform=preprocess
                                    )
train_data, val_data = torch.utils.data.random_split(data, [.8,.2])

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False, num_workers=num_workers)

In [ ]:
X, y = next(iter(train_dataloader))
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
y

In [ ]:
preprocess = net.weights.transforms()

In [ ]:
img_transformed = preprocess(X)

In [ ]:
def training_loop(net, trainloader, valloader, gpu=False, epochs=1):
    if gpu == False:
        device = torch.device("cpu")
    elif gpu == True:
        device = torch.device("mps")
    net = net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(epochs): 
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = net(inputs)
            train_loss = criterion(outputs, labels)
            train_loss.backward()
            optimizer.step()
            
            # print statistics
            #print(f"{train_loss.item()}")
            running_loss += train_loss.item()
            if i % 100 == 99:  # print every 200 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.8f}')
                running_loss = 0.0
        #train_acc = accuracy_score(net, trainloader, gpu=gpu)
        #val_acc = accuracy_score(net, valloader, gpu=gpu)
        print(f"Epoch {epoch} train_acc: {train_acc}, val_acc: {val_acc}")
    PATH = './models/net.pth'
    torch.save(net.state_dict(), PATH)
    print(f'Training complete - model saved to {PATH}')

In [ ]:
training_loop(net, train_dataloader, val_dataloader, gpu=gpu, epochs=epochs)